In [1]:
def clearall():
    all = [var for var in globals() if var[0] != "_"]
    for var in all:
        del globals()[var]
clearall()

In [2]:
print('Importing Librairies...')
import talib
import numpy as np
import pandas as pd
import pandas_datareader as web
from colorama import Fore, Back, Style
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import xgboost as xgb
from xgboost import XGBRegressor, plot_importance
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn')
import time
import datetime as dt
import os
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score,roc_curve,confusion_matrix,classification_report,r2_score
import warnings
import pickle
warnings.filterwarnings("ignore")
#import tensorflow as tf
#tf.logging.set_verbosity(tf.logging.ERROR)
print('Librairies imported')
print('')

___Author___='LumberJack Jyss'
print('Global Optimized LumberJack Environment Motor 55\nLumberJack Jyss 5779(c)')
print(Fore.BLUE,'°0Oo_D.A.G._26_oO0°')
print('DEEP LEARNING FOREX V1',Style.RESET_ALL)

#LaDate = input('Date de DL - YYYY-MM-DD ')
LaDate = '2020-01-15'
ticker = 'USDJPY_1H'
indice = 0.01
try:
    os.mkdir('DL_'+ticker+'-'+LaDate)
except:
    pass
try:
    os.mkdir('Boost_'+ticker+'-'+LaDate)
except:
    pass

Importing Librairies...


Using TensorFlow backend.


Librairies imported

Global Optimized LumberJack Environment Motor 55
LumberJack Jyss 5779(c)
 °0Oo_D.A.G._26_oO0°
DEEP LEARNING FOREX V1 


In [3]:
def prepa_data(df):
    rsi = talib.RSI(df['Close'],timeperiod=14)
    stoc_slowk, stoc_slowd = talib.STOCH(df['High'],df['Low'],df['Close'])
    upper, middle, lower =  talib.BBANDS(df['Close'], timeperiod=9, nbdevup=2, nbdevdn=2,matype=0)
    sma5 = talib.SMA(df['Close'],timeperiod=5)
    sma8 = talib.SMA(df['Close'],timeperiod=8)
    sma10 = talib.SMA(df['Close'],timeperiod=10)
    sma12 = talib.SMA(df['Close'],timeperiod=12)
    sma15 = talib.SMA(df['Close'],timeperiod=15)
    sma30 = talib.SMA(df['Close'],timeperiod=30)
    sma35 = talib.SMA(df['Close'],timeperiod=35)
    sma40 = talib.SMA(df['Close'],timeperiod=40)
    sma45 = talib.SMA(df['Close'],timeperiod=45)
    sma50 = talib.SMA(df['Close'],timeperiod=50)
    atr = talib.ATR(df['High'],df['Low'],df['Close'],timeperiod=10)
    delta5_8 = sma5 - sma8
    delta8_10 = sma8 - sma10
    delta10_12 = sma10 - sma12
    delta12_15 = sma12 - sma15
    delta15_30 = sma15 - sma30
    delta30_35 = sma30 - sma35
    delta35_40 = sma35 - sma40
    delta40_45 = sma40 - sma45
    delta45_50 = sma45 - sma50
    bbdelta = upper - middle
    price_bolup = df['Close'] - lower
    price_bolow = df['Close'] - upper
    Ema = talib.EMA(df['Close'],timeperiod=20)
    KC_High = Ema + 2*atr
    KC_Low = Ema - 2*atr
    aroondown, aroonup = talib.AROON(df['High'], df['Low'], timeperiod=9)
    aroon = aroonup - aroondown 
    rsi30_list = []
    rsi70_list = []
    for i in range(0,df.shape[0]):
        try:
            rsi70_list.append(70 - rsi[i])
            rsi30_list.append(rsi[i] - 30)
        except:
            rsi70_list.append(0)
            rsi30_list.append(0)
        
    varop_spy = df['Open'] - df['Close']
    varhl_spy = df['High'] - df['Low']
    df['Varop_Spy'] = varop_spy
    df['Varhl_spy'] = varhl_spy
    df['RSI'] = rsi
    df['70 - RSI'] = np.array(rsi70_list)
    df['RSI - 30'] = np.array(rsi30_list)
    df['BBD_Delta_Up'] = bbdelta
    df['delta5_8'] = delta5_8
    df['delta8_10'] = delta8_10
    df['delta10_12'] = delta10_12
    df['delta12_15'] = delta12_15
    df['delta15_30'] = delta15_30
    df['delta30_35'] = delta30_35
    df['delta35_40'] = delta35_40
    df['delta40_45'] = delta40_45
    df['delta45_50'] = delta45_50
    df['Stoc_Slowk'] = stoc_slowk
    df['Stoc_Slowd'] = stoc_slowd
    df['KC_High'] = KC_High
    df['KC_Low'] = KC_Low
    df['upper'] = upper
    df['lower'] = lower
    df['var_bollup_kchigh'] = upper-KC_High
    df['var_bolllow_kclow'] = lower-KC_Low
    df['Aroon Up'] = aroonup
    df['Aroon Down'] = aroondown
    df['Delta Aroon'] = aroon
    up = []
    down = []
    df = df.dropna()
    df['%Futur1'] = ((df['Close'].shift(-1)-df['Close']) *100) / (df['Close'])
    df['%Futur2'] = ((df['Close'].shift(-2)-df['Close']) *100) / (df['Close'])
    df['%Futur3'] = ((df['Close'].shift(-3)-df['Close']) *100) / (df['Close'])
    df['%Futur4'] = ((df['Close'].shift(-4)-df['Close']) *100) / (df['Close'])
    df['%Futur5'] = ((df['Close'].shift(-5)-df['Close']) *100) / (df['Close'])
    for i in range(0,df.shape[0]):
        if df.iloc[i]['%Futur1'] > indice or df.iloc[i]['%Futur2'] > indice or df.iloc[i]['%Futur3'] > indice or df.iloc[i]['%Futur4'] > indice or df.iloc[i]['%Futur5'] > indice:
            up.append(1)
            down.append(0)
        elif df.iloc[i]['%Futur1'] < -indice or df.iloc[i]['%Futur2'] < -indice or df.iloc[i]['%Futur3'] < -indice or df.iloc[i]['%Futur4'] < -indice or df.iloc[i]['%Futur5'] < -indice:
            up.append(0)
            down.append(1)
        else:
            up.append(0)
            down.append(0)
        
    df['target_up'] = up  
    df['target_down'] = down 
    #df = df.dropna()
    
    df = df[['High','Low','Open','Volume','DeltaVol','Close','Varop_Spy','Varhl_spy','RSI',\
             '70 - RSI','RSI - 30','BBD_Delta_Up','delta5_8','delta8_10','delta10_12',\
             'delta12_15','delta15_30','delta30_35','delta35_40','delta40_45','delta45_50',\
             'Stoc_Slowk','Stoc_Slowd','KC_High','KC_Low','upper','lower','var_bollup_kchigh',\
             'var_bolllow_kclow','Aroon Up','Aroon Down','Delta Aroon',\
             '%Futur1','%Futur2','%Futur3','%Futur4','%Futur5','target_up','target_down']]
        
    return(df)
    



def save_model(model_up,model_down):
    savename = 'DL_'+ticker+'-'+LaDate+'/Save_'+ticker
    # serialize model to YAML
    model_up_yaml = model_up.to_yaml()
    model_down_yaml = model_down.to_yaml()
    with open(savename+"_up.yaml", "w") as yaml_file:
        yaml_file.write(model_up_yaml)
    with open(savename+"_down.yaml", "w") as yaml_file:
        yaml_file.write(model_down_yaml)
    # serialize weights to HDF5
    model_up.save_weights(savename+"_up.h5")
    model_down.save_weights(savename+"_down.h5")

In [4]:
# PARAMETRES TEMPORELS INITIAUX
start = '2019-06-01'
end = LaDate
print('DATE ORIGINELLE DU DEBUT DE TOUT... :',start)
error = []

        
########################
#### MAIN SKAN55 #######
########################

print(' - Date de début : ',start,' - Date de fin : ',end)
print(Fore.YELLOW,'GENERATION DE LaBase :',Fore.MAGENTA,time.strftime("%H:%M:%S", time.localtime()))
tmps55=time.time()
LaBase = pd.read_csv('USDJPY_1H.csv')
df = LaBase

INDEX = []
for i in range(0,len(df)):
    INDEX.append((str(df.iloc[i,0])[6:10]+'-'+str(df.iloc[i,0])[3:5]+'-'+str(df.iloc[i,0])[:2]))
df.index = INDEX
df = df.drop(['Local time'],axis=1)
df = df[df.index>=start]
delta = round(df.shape[0])
bloc1 = round(delta*0.75)
bloc2 = delta - bloc1

tmps22=round(time.time()-tmps55,2)
print(Fore.BLUE,"Time for complete Base Preparation = %f" %tmps22,'seconds \n',Style.RESET_ALL)
#LaBasePrep.to_csv('LaBasePrep.csv')
df.head()

DATE ORIGINELLE DU DEBUT DE TOUT... : 2019-06-01
 - Date de début :  2019-06-01  - Date de fin :  2020-01-15
 GENERATION DE LaBase :  14:47:45
 Time for complete Base Preparation = 3.150000 seconds 
 


,Open,High,Low,Close,Volume
2019-06-02,108.275,108.348,108.266,108.320,1506.0699
2019-06-03,108.319,108.324,108.173,108.197,5449.0898
2019-06-03,108.199,108.300,108.197,108.223,5235.0898
2019-06-03,108.223,108.319,108.208,108.293,9715.6699
2019-06-03,108.292,108.302,108.150,108.232,11496.0400


In [5]:
###### PREPARATION #########


print('The computing has begun at ',dt.datetime.now().hour,':',dt.datetime.now().minute,':',dt.datetime.now().second)

BasePrep = pd.DataFrame()
tmps55=time.time()

df['DeltaVol'] = df['Volume'] - df['Volume'].shift(1)
df = df.dropna()
df = prepa_data(df)
    
BasePrep = df
tmps22=round(time.time()-tmps55,2)
print(Fore.BLUE,"\nTime for complete Preparation = %f" %tmps22,'seconds \n',Style.RESET_ALL)
BasePrep.to_csv('BasePrep.csv')
df.head()

The computing has begun at  14 : 47 : 49
 
Time for complete Preparation = 3.370000 seconds 
 


,High,Low,Open,Volume,DeltaVol,Close,Varop_Spy,Varhl_spy,RSI,70 - RSI,...,Aroon Up,Aroon Down,Delta Aroon,%Futur1,%Futur2,%Futur3,%Futur4,%Futur5,target_up,target_down
2019-06-05,108.270,108.195,108.202,3049.4299,-48.3401,108.257,-0.055,0.075,56.813746,0,...,11.111111,44.444444,-33.333333,-0.109000,-0.174585,-0.182898,-0.142254,-0.153339,0,1
2019-06-05,108.257,108.113,108.257,7804.3901,4754.9602,108.139,0.118,0.144,50.467131,0,...,0.000000,33.333333,-33.333333,-0.065656,-0.073979,-0.033290,-0.044387,-0.063807,0,1
2019-06-05,108.173,108.029,108.139,7637.2100,-167.1801,108.068,0.071,0.144,47.060637,0,...,0.000000,22.222222,-22.222222,-0.008328,0.032387,0.021283,0.001851,0.038864,1,0
2019-06-05,108.098,108.038,108.068,5721.5801,-1915.6299,108.059,0.009,0.060,46.630960,0,...,66.666667,11.111111,55.555556,0.040718,0.029613,0.010180,0.047196,0.172128,1,0
2019-06-05,108.135,108.044,108.058,4211.7700,-1509.8101,108.103,-0.045,0.091,49.078772,0,...,55.555556,0.000000,55.555556,-0.011101,-0.030526,0.006475,0.131356,0.133206,1,0


In [6]:
def boost(df,period):
    print(Fore.YELLOW,'\nGENERATION DU BOOST :',Fore.MAGENTA,time.strftime("%H:%M:%S", time.localtime()))
    tmps55=time.time()
    
    seed = 770
    np.random.seed(seed)
    X = df.copy()
    X = X.drop(['Close'],axis=1)
    X['Close'] = df['Close']
    y = X.iloc[:,-1]
    Xtrain = X.iloc[:bloc1,:-1]
    Xtest = X.iloc[bloc1:-period,:-1]
    yshift = y.shift(-period)
    ytrain = yshift.iloc[:bloc1]
    ytest = yshift.iloc[bloc1:-period]

    model = xgb.XGBRegressor(n_estimators=200000, learning_rate=1, gamma=1, subsample=1, colsample_bytree=1, max_depth=5,objective='reg:squarederror')

    model.fit( Xtrain, ytrain, early_stopping_rounds=10, eval_metric=['mae'],eval_set=[(Xtest, ytest)], verbose=0)

    ytrain_pred = model.predict(Xtrain)

    y_pred = model.predict(Xtest)

    pred = model.predict(X.iloc[:,:-1])
    r2 = r2_score(ytest,y_pred)*100
    df['Boost'+str(period)] = pred
    return(df,model,r2)

for period in range(0,5):
    period = period + 1
    df,model,r2 = boost(df,period)

    print(Fore.GREEN,'\nPeriod = ',period,'   ----   r2_score = ',r2,'%')

    print('% error : ',abs(((df['Boost'+str(period)].shift(+period)-df['Close'])/df['Close']*100)).mean(),'%')

    print('\nEcart min : ',abs(((df['Boost'+str(period)].shift(+period)-df['Close'])/df['Close']*100)).min(),'%')

    print('\nEcart max : ',abs(((df['Boost'+str(period)].shift(+period)-df['Close'])/df['Close']*100)).max(),'%')

    print('\n',model.evals_result())

    pickle.dump(model, open('Boost_'+ticker+'-'+LaDate+'/'+ticker+'_boost'+str(period)+'.dat', "wb"))

tmps22=round(time.time()-tmps55,2)
print(Fore.BLUE,"\nTime for complete BOOST GENERATION = %f" %tmps22,'seconds \n',Style.RESET_ALL)

df['Boost1'] = df['Boost1'] - df ['Close']
df['Boost2'] = df['Boost2'] - df ['Close']
df['Boost3'] = df['Boost3'] - df ['Close']
df['Boost4'] = df['Boost4'] - df ['Close']
df['Boost4'] = df['Boost5'] - df ['Close']

df = df[['High','Low','Open','Volume','DeltaVol','Close','Varop_Spy','Varhl_spy','RSI',\
             '70 - RSI','RSI - 30','BBD_Delta_Up','delta5_8','delta8_10','delta10_12',\
             'delta12_15','delta15_30','delta30_35','delta35_40','delta40_45','delta45_50',\
             'Stoc_Slowk','Stoc_Slowd','KC_High','KC_Low','upper','lower','var_bollup_kchigh',\
             'var_bolllow_kclow','Aroon Up','Aroon Down','Delta Aroon',\
             'Boost1','Boost2','Boost3','Boost4','Boost5','%Futur1','%Futur2','%Futur3','%Futur4','%Futur5','target_up','target_down']]

df = df.dropna()

 
GENERATION DU BOOST :  14:47:52
 
Period =  1    ----   r2_score =  42.77377305076344 %
% error :  0.1110723427979384 %

Ecart min :  1.7449230326732295e-06 %

Ecart max :  1.047000131459823 %

 {'validation_0': {'mae': [1.427006, 0.294422, 0.28933, 0.292095, 0.289685, 0.2897, 0.2897, 0.2897, 0.2897, 0.2897, 0.2897, 0.2897]}}
 
GENERATION DU BOOST :  14:47:53
 
Period =  2    ----   r2_score =  45.00966421562207 %
% error :  0.11631847683685452 %

Ecart min :  8.513365608119991e-07 %

Ecart max :  1.0376432771793727 %

 {'validation_0': {'mae': [1.426929, 0.304663, 0.297393, 0.288623, 0.285255, 0.285255, 0.285255, 0.285255, 0.285255, 0.285255, 0.285255, 0.285255, 0.285255, 0.285255]}}
 
GENERATION DU BOOST :  14:47:53
 
Period =  3    ----   r2_score =  60.674835919325055 %
% error :  0.11284763726209913 %

Ecart min :  3.571834148078835e-05 %

Ecart max :  0.964073986130072 %

 {'validation_0': {'mae': [1.426863, 0.311574, 0.299862, 0.23768, 0.235241, 0.235164, 0.235164, 0.235164, 0

In [7]:
df.shape,df['target_up'].sum(),df['target_down'].sum(),'Futur%',df['%Futur1'].mean(),df['%Futur2'].mean(),df['%Futur3'].mean(),df['%Futur4'].mean(),df['%Futur5'].mean(),\
'Boost',df['Boost1'].mean(),df['Boost2'].mean(),df['Boost3'].mean(),df['Boost4'].mean(),df['Boost5'].mean(),

((3905, 44),
 2783,
 1089,
 'Futur%',
 0.00046941887789655437,
 0.0009663123239351865,
 0.0014828083619448484,
 0.002002037015613941,
 0.002498106974809598,
 'Boost',
 -0.061934910937125234,
 -0.05936169492987788,
 -0.046331048560539684,
 -0.060926308887594365,
 108.0477523803711)

In [8]:
df.tail()

,High,Low,Open,Volume,DeltaVol,Close,Varop_Spy,Varhl_spy,RSI,70 - RSI,...,Boost3,Boost4,Boost5,%Futur1,%Futur2,%Futur3,%Futur4,%Futur5,target_up,target_down
2020-01-17,110.158,110.100,110.149,1985.5400,-519.3800,110.101,0.048,0.058,46.700872,0,...,-0.857142,-0.993242,109.107758,0.077202,0.045413,0.059037,0.054495,0.035422,1,0
2020-01-17,110.212,110.093,110.099,4875.6400,2890.1000,110.186,-0.087,0.119,55.653815,0,...,-0.942142,-1.078242,109.107758,-0.031764,-0.018151,-0.022689,-0.041748,-0.044470,0,1
2020-01-17,110.212,110.103,110.186,7116.7699,2241.1299,110.151,0.035,0.109,51.795716,0,...,-0.907142,-1.043242,109.107758,0.013618,0.009078,-0.009986,-0.012710,-0.002724,1,0
2020-01-17,110.183,110.047,110.150,6643.3800,-473.3899,110.166,-0.016,0.136,53.290210,0,...,-0.922142,-1.058242,109.107758,-0.004539,-0.023601,-0.026324,-0.016339,-0.011800,0,1
2020-01-17,110.201,110.130,110.165,6862.9200,219.5400,110.161,0.004,0.071,52.703650,0,...,-0.917142,-1.121876,109.039124,-0.019063,-0.021786,-0.011801,-0.007262,-0.053558,0,1


In [ ]:
def deep_learning_down(df):
    
    seed = 770
    np.random.seed(seed)
    X = df.iloc[:,:-7]
    y_down = df.iloc[:,-1].values
    X.astype(np.float64)
    scaler = MinMaxScaler(feature_range=(0, 1))
    X = scaler.fit_transform(X)
    y_down = np.array(y_down).reshape(-1,1)

    Xtrain = X[:bloc1,:]
    Xtest = X[bloc1:,:]

    ytrain_down = y_down[:bloc1,:]
    ytest_down = y_down[bloc1:,:]

    ytrain_down = ytrain_down.reshape(ytrain_down.shape[0],)

    Xtrain = Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1])
        
    model_down = Sequential()
    # Add an input layer 
    model_down.add(Dense(50, activation='relu'))
    # Add an input layer 
    model_down.add(Dense(50, activation='relu'))
    # Add an input layer 
    model_down.add(Dense(50, activation='relu'))
    # Add an input layer 
    model_down.add(Dense(50, activation='relu'))
    # Add one hidden layer 
    model_down.add(Dense(23, activation='relu'))
    # Add an output layer 
    model_down.add(Dense(1, activation='sigmoid'))
    
    model_down.compile(loss='binary_crossentropy',
                  optimizer='adam', #rmsprop
                  metrics=['accuracy','mse'])

    history_down = model_down.fit(Xtrain, ytrain_down,epochs=600, batch_size=300, verbose=0)
    

    train_acc_down = model_down.evaluate(Xtrain, ytrain_down,verbose=1)

    yhat_down = model_down.predict_classes(Xtest)

    score_down = model_down.evaluate(Xtest, ytest_down,verbose=1)

    predict_down = model_down.predict(Xtest)

    accuracy_down = accuracy_score(ytest_down, yhat_down)

    # La précision permet de mesurer la capacité du modèle à refuser résultats non-pertinents : vrais_positifs/(vrais_positifs+faux_positifs)
    precision_down = precision_score(ytest_down, yhat_down) 


    # Recall : (vrai_positifs/(vrais_positifs+faux_négatifs))
    recall_down = recall_score(ytest_down, yhat_down) 


    resultats = pd.DataFrame()
    resultats['Date'] = df.index[bloc1:]
    resultats.index= df.index[bloc1:]
   
    resultats['Move Down'] = yhat_down
    resultats['Confiance Down'] = (predict_down)*100
    resultats['Actual'] = df.iloc[bloc1:]['Close']
    resultats['Actual.S'] = df.iloc[bloc1:]['Close'].shift(-1)
    open_S = df['Open'].shift(-1)
    resultats['Open.S'] = open_S.iloc[bloc1:]
    
    
    return(resultats,precision_down,model_down,scaler)

def deep_learning_up(df):
    
    seed = 770
    np.random.seed(seed)
    X = df.iloc[:,:-7]
    y_up = df.iloc[:,-2].values
    X.astype(np.float64)
    scaler = MinMaxScaler(feature_range=(0, 1))
    X = scaler.fit_transform(X)
    y_up = np.array(y_up).reshape(-1,1)

    Xtrain = X[:bloc1,:]
    Xtest = X[bloc1:,:]
    ytrain_up = y_up[:bloc1,:]
    ytest_up = y_up[bloc1:,:]
    

    ytrain_up = ytrain_up.reshape(ytrain_up.shape[0],)

    Xtrain = Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1])

    model_up = Sequential()
    
    # Add an input layer 
    model_up.add(Dense(50, activation='relu'))
    # Add an input layer 
    model_up.add(Dense(50, activation='relu'))
    # Add an input layer 
    model_up.add(Dense(50, activation='relu'))
    # Add an input layer 
    model_up.add(Dense(50, activation='relu'))
    # Add one hidden layer 
    model_up.add(Dense(23, activation='relu'))
    # Add an output layer 
    model_up.add(Dense(1, activation='sigmoid'))


    model_up.compile(loss='binary_crossentropy',
                  optimizer='adam', #rmsprop
                  metrics=['accuracy','mse'])
    
              

    history_up = model_up.fit(Xtrain, ytrain_up,epochs=600, batch_size=300, verbose=0)
            
    
    

    train_acc_up = model_up.evaluate(Xtrain, ytrain_up,verbose=1)

    yhat_up = model_up.predict_classes(Xtest)

    score_up = model_up.evaluate(Xtest, ytest_up,verbose=1)

    predict_up = model_up.predict(Xtest)

    accuracy_up = accuracy_score(ytest_up, yhat_up)

    # La précision permet de mesurer la capacité du modèle à refuser résultats non-pertinents : vrais_positifs/(vrais_positifs+faux_positifs)
    precision_up = precision_score(ytest_up, yhat_up)  


    # Recall : (vrai_positifs/(vrais_positifs+faux_négatifs))
    recall_up = recall_score(ytest_up, yhat_up) 


    resultats['Move Up'] = yhat_up
    resultats['Confiance up'] = (predict_up)*100

    dmp_cp=[]
    dmp_cp = ((resultats['Confiance up']-resultats['Confiance Down'])/(resultats['Confiance up']+resultats['Confiance Down'])*100)
    resultats['DMP_CP'] = dmp_cp
    return(resultats,precision_up,model_up,scaler)




tmps55=time.time()
print(Fore.BLUE,'\nDeeping in blue for ',ticker,Style.RESET_ALL)

resultats,precision_down,model_down,scaler = deep_learning_down(df)

resultats,precision_up,model_up,scaler = deep_learning_up(df)

n = 0    
while (precision_down*100) < 69:
    n+=1
    print('\nTicker : ',ticker,' Precision Down',precision_down*100,'\n')
    print('Test Raté ',n,' fois. On remet cela. \n')
    tmps5=time.time()
    resultats,precision_down,model_down,scaler = deep_learning_down(df)
    print(round(time.time()-tmps5,2),' secondes')
    
# serialize model to YAML
model_down_yaml = model_down.to_yaml()
savename = 'DL_'+ticker+'-'+LaDate+'/Save_'+ticker
with open(savename+"_down.yaml", "w") as yaml_file:
    yaml_file.write(model_down_yaml)
# serialize weights to HDF5
model_down.save_weights(savename+"_down.h5")

print('Le ',Fore.BLUE,'Deep Learning',Style.RESET_ALL ,'de ',Fore.YELLOW,ticker,Style.RESET_ALL,' pour le DOWN a été effecué avec succès. Les modèles ont été sauvegardés')

n = 0
while (precision_up*100) < 69:
    n+=1
    print('\nTicker : ',ticker,'Precision Up :',precision_up*100,'\n')
    print('Test Raté ',n,' fois. On remet cela. \n')
    tmps5=time.time()
    resultats,precision_up,model_up,scaler = deep_learning_up(df)
    print(round(time.time()-tmps5,2),' secondes')


savename = 'DL_'+ticker+'-'+LaDate+'/Save_'+ticker
# serialize model to YAML
model_up_yaml = model_up.to_yaml()
with open(savename+"_up.yaml", "w") as yaml_file:
    yaml_file.write(model_up_yaml)
# serialize weights to HDF5
model_up.save_weights(savename+"_up.h5")

print('Le ',Fore.BLUE,'Deep Learning',Style.RESET_ALL ,'de ',Fore.YELLOW,ticker,Style.RESET_ALL,' pour le UP a été effecué avec succès. Les modèles ont été sauvegardés')


    
print(Fore.YELLOW,Back.MAGENTA,Style.DIM,'PASSAGE FINI!!!!!!',Style.RESET_ALL)
tmps2=round(time.time()-tmps55,2)
print ("Job done in = %f" %tmps2,'seconds')

 
Deeping in blue for  USDJPY_1H 
935/935 [==============================] - 0s 25us/step

Ticker :  USDJPY_1H  Precision Down 35.051546391752574 

Test Raté  1  fois. On remet cela. 

935/935 [==============================] - 0s 21us/step
13.03  secondes

Ticker :  USDJPY_1H  Precision Down 33.27067669172932 

Test Raté  2  fois. On remet cela. 

935/935 [==============================] - 0s 18us/step
12.52  secondes

Ticker :  USDJPY_1H  Precision Down 37.10021321961621 

Test Raté  3  fois. On remet cela. 

935/935 [==============================] - 0s 21us/step
12.82  secondes

Ticker :  USDJPY_1H  Precision Down 33.778625954198475 

Test Raté  4  fois. On remet cela. 

935/935 [==============================] - 0s 18us/step
12.89  secondes

Ticker :  USDJPY_1H  Precision Down 32.41379310344827 

Test Raté  5  fois. On remet cela. 

935/935 [==============================] - 0s 21us/step
12.1  secondes

Ticker :  USDJPY_1H  Precision Down 34.387351778656125 

Test Raté  6  fois. O

In [ ]:
precision_down,precision_up

In [ ]:
from sklearn import  metrics, model_selection
from xgboost.sklearn import XGBClassifier

In [ ]:
%%time
seed = 770
np.random.seed(seed)
X = df.iloc[:,:-7]
y_up = df.iloc[:,-2].values
y_down = df.iloc[:,-1].values
X.astype(np.float64)
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
'''y_up = np.array(y_up).reshape(-1,1)
y_down = np.array(y_down).reshape(-1,1)'''

Xtrain = X[:bloc1,:]
Xtest = X[bloc1:,:]
ytrain_up = y_up[:bloc1]
ytest_up = y_up[bloc1:]
ytrain_down = y_down[:bloc1]
ytest_down = y_down[bloc1:]


model_up = XGBClassifier(
                            max_depth=80,
                            learning_rate=1,
                            n_estimators=1000000,
                            verbosity=1,
                            silent=None,
                            objective='binary:logistic',
                            booster='gbtree',
                            n_jobs=1,
                            nthread=None,
                            gamma=1,
                            min_child_weight=1,
                            max_delta_step=1,
                            subsample=1,
                            colsample_bytree=1,
                            colsample_bylevel=1,
                            colsample_bynode=1,
                            reg_alpha=1,
                            reg_lambda=1,
                            scale_pos_weight=1,
                            base_score=0.99,
                            random_state=1,
                            seed=None,
                            missing=None,).fit(Xtrain, ytrain_up)

model_down = XGBClassifier(
                            max_depth=80,
                            learning_rate=1,
                            n_estimators=100,
                            verbosity=1,
                            silent=None,
                            objective='binary:logistic',
                            booster='gbtree',
                            n_jobs=1,
                            nthread=None,
                            gamma=1,
                            min_child_weight=1,
                            max_delta_step=1,
                            subsample=1,
                            colsample_bytree=1,
                            colsample_bylevel=1,
                            colsample_bynode=1,
                            reg_alpha=1,
                            reg_lambda=1,
                            scale_pos_weight=1,
                            base_score=0.99,
                            random_state=1,
                            seed=None,
                            missing=None,).fit(Xtrain, ytrain_up)

# use the model to make predictions with the test data

y_pred_up = model_up.predict(Xtest)
y_pred_down = model_down.predict(Xtest)
# how did our model perform?

count_misclassified = (ytest_up != y_pred_up).sum()

print('Misclassified samples: {}'.format(count_misclassified))


#yhat_up = model_up.predict_classes(Xtest)
#yhat_down = model_down.predict_classes(Xtest)

#score_up = model_up.evaluate(Xtest, ytest_up,verbose=1)
#score_down = model_down.evaluate(Xtest, ytest_down,verbose=1)

predict_up = model_up.predict(Xtest)
predict_down = model_down.predict(Xtest)

accuracy_up = accuracy_score(ytest_up, y_pred_up)
accuracy_down = accuracy_score(ytest_down, y_pred_down)

# La précision permet de mesurer la capacité du modèle à refuser résultats non-pertinents : vrais_positifs/(vrais_positifs+faux_positifs)
precision_up = precision_score(ytest_up, y_pred_up)  
precision_down = precision_score(ytest_down, y_pred_down) 


# Recall : (vrai_positifs/(vrais_positifs+faux_négatifs))
recall_up = recall_score(ytest_up, y_pred_up) 
recall_down = recall_score(ytest_down, y_pred_down)  



print('Accuracy UP: {:.2f}'.format(accuracy_up*100))
print('Accuracy DOWN: {:.2f}'.format(accuracy_down*100))
#print('SCORE UP: {:.2f}'.format(score_up*100))
#print('SCORE DOWN: {:.2f}'.format(score_down*100))
print('Precision  UP: {:.2f}'.format(precision_up*100))
print('Precision DOWN: {:.2f}'.format(precision_down*100))
print('Recall UP: {:.2f}'.format(recall_up*100))
print('Recall DOWN: {:.2f}'.format(recall_down*100))



In [ ]:
if (precision_up*100) < 69 or (precision_down*100) < 69:
    print('\nTicker : ',ticker,'Precision Up :',precision_up*100,' ---- Precision Down',precision_down*100,'\n')
    print('Test Raté 1 fois. On remet cela. \n')
    tmps5=time.time()
    resultats,precision_up,precision_down,model_up,model_down,scaler = deep_learning(df)
    print(round(time.time()-tmps5,2),' secondes')
    
if (precision_up*100) < 69 or (precision_down*100) < 69:
    print('\nTicker : ',ticker,'Precision Up :',precision_up*100,' ---- Precision Down',precision_down*100,'\n')
    print('Test Raté 2 fois. On remet cela. \n')
    tmps5=time.time()
    resultats,precision_up,precision_down,model_up,model_down,scaler = deep_learning(df)
    print(round(time.time()-tmps5,2),' secondes')
    
if (precision_up*100) < 69 or (precision_down*100) < 69:
    print('\nTicker : ',ticker,'Precision Up :',precision_up*100,' ---- Precision Down',precision_down*100,'\n')
    print('Test Raté 3 fois. \n')
    print('Le ',Fore.BLUE,'Deep Learning',Style.RESET_ALL ,'de ',Fore.YELLOW,ticker,Style.RESET_ALL,' a échoué. Les modèles ne sont pas sauvegardés')
    print(Fore.YELLOW,Back.MAGENTA,Style.DIM,'PASSAGE FINI!!!!!!',Style.RESET_ALL)
    tmps2=round(time.time()-tmps55,2)
    print ("Job done in = %f" %tmps2,'seconds')

if (precision_up*100) >= 69 and (precision_down*100) >= 69:
    save_model(model_up, model_down)

    print('Le ',Fore.BLUE,'Deep Learning',Style.RESET_ALL ,'de ',Fore.YELLOW,ticker,Style.RESET_ALL,' a été effecué avec succès. Les modèles ont été sauvegardés')
    print(Fore.YELLOW,Back.MAGENTA,Style.DIM,'PASSAGE FINI!!!!!!',Style.RESET_ALL)
    tmps2=round(time.time()-tmps55,2)
    print ("Job done in = %f" %tmps2,'seconds')
    